In [1]:
import models
import torch
import pandas as pd
import utils

import carla
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from itertools import count
from PIL import Image

from torch.utils.tensorboard import SummaryWriter

import gym
import gym_carla

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
params = {
    'number_of_vehicles': 3,
    'number_of_walkers': 0,
    'display_size': 256,  # screen size of bird-eye render
    'max_past_step': 1,  # the number of past steps to draw
    'dt': 0.1,  # time interval between two frames
    'discrete': True,  # whether to use discrete control space
    'discrete_acc': [2.0],  # discrete value of accelerations
    'discrete_steer': [-0.3, 0.0, 0.3],  # discrete value of steering angles
    'continuous_accel_range': [-3.0, 3.0],  # continuous acceleration range
    'continuous_steer_range': [-0.3, 0.3],  # continuous steering angle range
    'ego_vehicle_filter': 'vehicle.lincoln*',  # filter for defining ego vehicle
    'port': 3000,  # connection port
    'town': 'Town04',  # which town to simulate
    'task_mode': 'random',  # mode of the task, [random, roundabout (only for Town03)]
    'max_time_episode': 500,  # maximum timesteps per episode
    'max_waypt': 12,  # maximum number of waypoints
    'obs_range': 32,  # observation range (meter)
    'lidar_bin': 0.125,  # bin size of lidar sensor (meter)
    'd_behind': 12,  # distance behind the ego vehicle (meter)
    'out_lane_thres': 1.8,  # threshold for out of lane
    'desired_speed': 3,  # desired speed (m/s)
    'max_ego_spawn_times': 20,  # maximum times to spawn ego vehicle
    'display_route': True,  # whether to render the desired route
    'pixor_size': 64,  # size of the pixor labels
    'pixor': False,  # whether to output PIXOR observation
    'use_fixed':True,
    'weather':'ClearNoon',
    'obs_size':128,
    'routes':None,
    'Collect_Data':False
}

# Set gym-carla environment
env = gym.make('carla-v0', params=params)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
connecting to Carla server...
Carla server connected!
WeatherParameters(cloudiness=5.000000, cloudiness=5.000000, precipitation=0.000000, precipitation_deposits=0.000000, wind_intensity=10.000000, sun_azimuth_angle=-1.000000, sun_altitude_angle=45.000000, fog_density=2.000000, fog_distance=0.750000, fog_falloff=0.100000, wetness=0.000000, scattering_intensity=1.000000, mie_scattering_scale=0.030000, rayleigh_scattering_scale=0.033100)


In [48]:
import time
def save_images(rgb,generated,step,folder):
    im = Image.fromarray(rgb)
    im.save('./Video/'+folder+'/RGB/'+str(step)+".jpeg")
    generated.save('./Video/'+folder+'/Sem/'+str(step)+'.jpeg')


In [74]:
vis_net = models.DeepDQN(3,device).to(device)
vis_net.load_state_dict(torch.load('./model_params_CL/model_10.final'))
vis_net.eval()


DeepDQN(
  (lin1): Linear(in_features=9, out_features=80, bias=True)
  (lin2): Linear(in_features=80, out_features=50, bias=True)
  (lin3): Linear(in_features=50, out_features=25, bias=True)
  (lin4): Linear(in_features=25, out_features=15, bias=True)
  (lin5): Linear(in_features=15, out_features=8, bias=True)
  (lin6): Linear(in_features=8, out_features=3, bias=True)
)

### Waypoint Model
#### Parameters

In [49]:
#test routes 192, 193, 197, 371
route = {'Town04':{'H':[192]}} 
env.routes = route
env.routes_dict = route
env.desired_speed = 5
env.max_time_episode = 100

In [83]:
num_episodes = 1
env.use_fixed = 'H'
env.route_idx = 1
levels = ['E','M','H']
next_lvl = 4
min_overall_loss = 1000
results = np.zeros((num_episodes,3))
for i_episode in range(num_episodes):
    rewards = 0
    # Initialize the environment and state
    obs = env.reset()
    #ego_dir retirves the distance and angle from vehicle to nearest waypoint
    ego_location = env.ego.get_location()
    ego_dir = gym_carla.envs.misc.get_lane_dis(env.waypoints,ego_location.x,ego_location.y)
    #pos gets a distanc d and array w which has to be seperated out in below line
    ego_pos = np.asarray((ego_dir[0],ego_dir[1][0],ego_dir[1][1]),dtype=np.float32)
    state = np.concatenate((ego_pos,np.zeros(6)))
    state = torch.tensor(state)
    # Resize, and add a batch dimension (BCHW)
    loss = episode_loss = 1000
    for t in count():
        # Select and perform an actionii#
        with torch.no_grad():
            action = vis_net(state.float()).argmax().view(1,1)
        obs, reward, done, info  = env.step(action.item())
        sem_image = utils.replace(obs['semantic'][:,:,0])
        env.show_images(sem_image)
        save_images(obs['camera'],Image.fromarray(sem_image,'RGB'),t,'Waypoint')
        rewards += reward
        
        #pos gets a distanc d and array w which has to be seperated out in below line
        pos = np.asarray((info['position'][0],info['position'][1][0],info['position'][1][1]))
        ang = np.asarray(info['angular_vel'])
        acc = np.asarray(info['acceleration'])
        steer = np.asarray(info['steer'])
        next_state = np.concatenate((pos, ang, acc, steer), axis=None)
        #update state
        state = torch.tensor(next_state)

        if done:
            results[i_episode,0] = rewards
            results[i_episode,1] = t
            if t  == env.max_time_episode:
                results[i_episode,2] = 1
            else:
                results[i_episode,2] = 0
            print(results)
            print('########')
            
            break


[[101. 100.   1.]]
########


In [59]:
obs['semantic'][:,:,0]

array([[13, 13, 13, ...,  9,  9,  9],
       [13, 13, 13, ...,  9,  9,  9],
       [13, 13, 13, ...,  9,  9,  9],
       ...,
       [ 7,  7,  7, ...,  7,  7,  7],
       [ 7,  7,  7, ...,  7,  7,  7],
       [ 7,  7,  7, ...,  7,  7,  7]], dtype=uint8)

In [3]:
def save_results(results,file):
    data = pd.DataFrame(results,columns=['Reward','Length','Completed'])
    data.to_csv(file)
    return data

    
    

In [29]:
out = save_results(results,'./model_results/SimpleModel/Clear/gentle_right.csv')
out

,Reward,Length,Completed
0,99.007892,100.0,1.0
1,99.395705,100.0,1.0
2,101.000000,100.0,1.0
3,98.326883,100.0,1.0
4,101.000000,100.0,1.0
5,100.307476,100.0,1.0
6,88.963381,100.0,1.0
7,93.951552,100.0,1.0
8,101.000000,100.0,1.0
9,101.000000,100.0,1.0


### AutoEncoder Model

In [67]:
vis_net = models.Full_DQN(3,64,device).to(device)
vis_net.load_state_dict(torch.load('./model_params_CL/Full_model_17.best'))
vis_net.eval()

model = models.PerceptionNet(device).to(device)
model.load_state_dict(torch.load('./AE_params/model_46.best'))
model.eval()


PerceptionNet(
  (conv1): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn5): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6a): Conv2d(512, 64, kernel_size=(4, 4), stride=(1, 1))
  (conv6b): Conv2d(512, 64, kernel_size=(4, 4), stride=(1, 1))
  (conv7): ConvTranspose2d(64, 512, kernel_size=(4, 4), stride=(1, 1))
  (bn6): BatchNorm2d(512, eps=1e-05, mom

In [70]:
#test routes 192, 193, 197, 371, 348
route = {'Town04':{'H':[193]}} 
env.routes = route
env.routes_dict = route
env.desired_speed = 5
env.max_time_episode = 100

In [72]:
num_episodes = 1
env.use_fixed = 'H'
env.route_idx = 1
levels = ['E','M','H']
next_lvl = 4
min_overall_loss = 1000
input_size = 73
results = np.zeros((num_episodes,3))
for i_episode in range(num_episodes):
    eps_loss = []
    rewards = 0
    # Initialize the environment and state
    obs = env.reset()
    ego_location = env.ego.get_location()
    ego_dir = gym_carla.envs.misc.get_lane_dis(env.waypoints,ego_location.x,ego_location.y)
    #pos gets a distanc d and array w which has to be seperated out in below line
    ego_pos = np.asarray((ego_dir[0],ego_dir[1][0],ego_dir[1][1]),dtype=np.float32)
    state = np.concatenate((ego_pos,np.zeros(6)))
    state = torch.tensor(state).reshape(1,9,1,1)

    new_obs = torch.tensor(obs['camera'])
    new_obs = new_obs.permute(2,0,1).reshape(1,3,128,128)
    
    _,latent_space = model(new_obs)
    state = torch.cat((state,latent_space.cpu()),1).reshape(1,input_size)
    # Resize, and add a batch dimension (BCHW)
    loss = episode_loss = 1000
    for t in count():
        # Select and perform an action
        
        with torch.no_grad():
            action = vis_net(state.float()).argmax().view(1,1)
            obs, reward, done, info  = env.step(action.item())
            sem_image = utils.generate_semantic_im(torch.tensor(obs['camera']).permute(2,0,1),model)
            env.show_images(np.asarray(sem_image))
            save_images(obs['camera'],sem_image,t,'Full')
            new_obs = torch.tensor(obs['camera'])
            new_obs = new_obs.permute(2,0,1).reshape(1,3,128,128)
            _,latent_space = model(new_obs)
            rewards += reward
            reward = torch.tensor([reward], device=device)

            #pos gets a distanc d and array w which has to be seperated out in below line
            pos = np.asarray((info['position'][0],info['position'][1][0],info['position'][1][1]))
            ang = np.asarray(info['angular_vel'])
            acc = np.asarray(info['acceleration'])
            steer = np.asarray(info['steer'])
            next_state = np.concatenate((pos, ang, acc, steer), axis=None)
            
            
            info_state = torch.tensor(next_state).reshape(1,9,1,1)
            next_state = torch.cat((info_state,latent_space.cpu()),1).reshape(1,input_size)

        state = next_state

        if done:
            results[i_episode,0] = rewards
            results[i_episode,1] = t
            if t  == env.max_time_episode:
                results[i_episode,2] = 1
            else:
                results[i_episode,2] = 0
            print(results)
            print('########')
            break

[[101. 100.   1.]]
########


In [46]:
out = save_results(results,'./model_results/FullModel/Clear/right_turn.csv')
out

,Reward,Length,Completed
0,101.0,100.0,1.0
1,101.0,100.0,1.0
2,101.0,100.0,1.0
3,101.0,100.0,1.0
4,101.0,100.0,1.0
5,101.0,100.0,1.0
6,101.0,100.0,1.0
7,101.0,100.0,1.0
8,101.0,100.0,1.0
9,101.0,100.0,1.0
